https://zhuanlan.zhihu.com/p/340149804

# Attention is All You Need

<div>
  <img src="https://pic4.zhimg.com/v2-1719966a223d98ad48f98c2e4d71add7_r.jpg" width="400"/>
</div>

## 总体技术脉络

1. 你需要的只是注意力

    - Attention is All You Need: 开山鼻祖 (Google) (NIPS2017)

2. Transformer 在目标检测任务的演进

    - End-to-End Object Detection with Transformers (DETR)：引入视觉领域的首创 DETR (ECCV2020)
    - Deformable DETR: 可变形的 Transformer (ICLR2021)
    - Swin Transformer for Object Detection: 层次化视觉 Transformer (MSRA) _SOTA_

3. Transformer 在识别任务的演进

    - Vision Transformer (ViT): 用于分类任务的 Transformer (ICLR2021)
    - Transformer + Distillation (DeiT): 高效图像 Transformer (FAIR)
    - Transformer Visual Recognition (Visual Transformers): 基于 Token 的图像表示和处理 (UC Berkeley)
    - Going deeper with Image Transformers (CaiT) (FAIR)
    - Introducing Convolutions to Vision Transformers (CvT): 将卷积融入视觉 Transformer (McGill, Microsoft)
    - Transformer iN Transformer (TNT): 充分挖掘 patch 内部信息 (华为诺亚)
    - Swin Transformer: 层次化视觉 Transformer (MSRA) _SOTA_

4. Transformer 内部机制的探究

    - Transformer iN Transformer (TNT): 充分挖掘 patch 内部信息 (华为诺亚)
    - Do We Really Need Explicit Position Encodings for Vision Transformers? 探究位置编码的必要性 (美团)

5. 轻量化 Transformer 探索

    - Efficient Transformer (HAT): 高效的硬件感知 Transformer (MIT-Han)
    - Efficient Transformer (Lite-Transformer): 远近注意力机制的轻量化Transformer (MIT-Han)
    - DeFINE: 深度矩阵分解给词向量矩阵瘦身 (华盛顿大学 ICLR2020)
    - LeViT: 用于快速推理的视觉 Transformer (FAIR)

6. 自监督视觉 Transformer

    - MoCo v3: 训练自监督视觉 Transformer 的实证研究 (FAIR)
    - DINO: 自监督视觉 Transformer 的新兴特性 (FAIR)
    - MoBY with Swin Transformer (MSRA)

7. 用于底层视觉任务的 Transformer

    - Transformer + Image Processing (IPT): 用于底层视觉任务的 Transformer (华为诺亚)

## 1. Transformer: 你需要的只是注意力

Transformer 是 Google 的机器翻译团队在 2017 年提出的一种 NLP 经典模型，现在比较火热的 Bert 也是基于 Transformer. Transformer 模型使用了 Self-Attention 机制，不采用 RNN 的顺序结构，使得模型可以并行化训练，而且能够拥有全局信息。

### 1.1 传统网络结构 (CNN/RNN) 的局限性

- RNN: 不容易并行化 (Hard to parallel)
- CNN: 视野有限, 不能充分利用上下文向量信息

### 1.2 Transformer 的出现

- Self-Attention: 每个输出可以看到完整的 Sequence, 同时可以并行化计算
- Positional Encoding: 引入位置编码向量来捕捉顺序序列的相对位置信息

### 1.3 Self-Attention 机制

那么 Self-Attention 具体是如何工作的呢？

<div>
  <img src="https://pic2.zhimg.com/v2-8628bf2c2bb9a7ee2c4a0fb870ab32b9_r.jpg" width="500"/>

  Fig. 1.1: Self-Attention 就是一堆矩阵乘法，可以实现 GPU 加速
</div>

撇开缩放因子不说, 那么 Attention 的运算可以分解为:

$$O=V \hat{A}, \hat{A} = \mathrm{softmax}{A}, A = K^{\top}Q$$

其中 $Q, K, V$ 是输入序列的变换. 即 Self-Attention 就是一堆矩阵乘法，可以使用 GPU 进行加速.

### 1.4 Visualizing Multi-head Self-Attention

<div>
  <img src="https://pic3.zhimg.com/80/v2-6b6c906cfca399506d324cac3292b04a_720w.jpg" width="500"/>

  Fig. 1.2: Multi-head Self-Attention 的不同 Head 分别关注了 Global 和 Local 的讯息
</div>

### 1.5 Positional Encoding: $\mathrm{PE}$

\begin{aligned}
\mathrm{PE}\left(\mathrm{pos}, 2i\right) &= \sin\left(\mathrm{pos} / 10000^{2i/d}\right) \\
\mathrm{PE}\left(\mathrm{pos}, 2i+1\right) &= \cos\left(\mathrm{pos} / 10000^{2i/d}\right)
\end{aligned}

在这种设计方式下, 下一个位置的编码向量可以由前面的编码向量线性表达, 等价于以一种非常容易学会的方式告诉了网络单词之间的绝对位置, 让模型能够轻松学习到相对位置信息.

### 1.6 Masked Multi-Head Self-Attention

每次解码都会且只会利用前面已经解码输出的所有单词嵌入信息, 同时又要使训练并行化, 为此就需要 Masked Multi-Head Self-Attention 模块. 具体是: 在解码器中，Self-Attention 层只被允许处理输出序列中更靠前的那些位置, 在 Softmax 步骤前, 它会把后面的位置给隐去.

<div>
  <img src="https://pic3.zhimg.com/v2-b0891e9352874c9eee469372b85ecbe2_r.jpg" width="500"/>

  Fig. 1.3: Self-Attention 的计算机制
</div>

这一步主要是由建立模型的时候按照顺序推理有序输出的逻辑限制的. 也称之为 **Predicts the output sequence one element at a time**.

## 2. ViT: Vision Transformer

### 2.1 图片预处理: 分块和降维

<div>
  <img src="https://pic2.zhimg.com/v2-7439a17c2e9aa981c95d783a93cb8729_r.jpg" width="500px">

  Fig. 2.1: An Image is Worth 16x16 Words
</div>

## 3. **DE⫶TR**: End-to-End Object Detection with Transformers

<div>
  <img src="https://gitee.com/zhiqwang/detr/raw/master/.github/DETR.png" width="800"/>

  Fig. 3.1: DETR directly predicts (in parallel) the final set of detections by combining
a common CNN with a transformer architecture. During training, bipartite matching
uniquely assigns predictions with ground truth boxes. Prediction with no match should
yield a “*no object*” (⦰) class prediction.
</div>

### 3.1 Set predictions

1. a set prediction loss that forces unique matching between predicted and ground truth boxes;
1. an architecture that predicts (in a single pass) a set of objects and models their relation.

### 3.2 模型结构

The overall DETR architecture is surprisingly simple and depicted in Figure 3.2. It contains three main components, which we describe below: a CNN backbone to extract a compact feature representation, an encoder-decoder transformer, and a simple feed forward network (FFN) that makes the final detection prediction.

<div>
  <img src="https://pic4.zhimg.com/v2-3d43474df51c545ad6bafc19b3c8ccc3_r.jpg" width="800"/>

  Fig. 3.2: DETR uses a conventional CNN backbone to learn a 2D representation of an input image. The model flattens it and supplements it with a positional encoding before passing it into a transformer encoder. A transformer decoder then takes as input a small fixed number of learned positional embeddings, which we call *object queries*, and additionally attends to the encoder output. We pass each output embedding of the
decoder to a shared feed forward network (FFN) that predicts either a detection (class and bounding box) or a “*no object*” class.
</div>

<div>
  <img src="https://pic3.zhimg.com/v2-c158521c7a602382dfa4d85243672df2_r.jpg" width="600"/>

  Fig. 3.3: DETR 使用的 Transformer 结构
</div>

- 第一个是用 transformer 的 encoder-decoder 架构一次性生成 $N$ 个 box prediction. 其中 $N$ 是一个事先设定的、并多于图像中 Object 个数的一个整数。
- 第二个是设计了 bipartite matching loss，基于预测的 Boxes 和 Ground truth Boxes 的二分图匹配计算 loss 的大小，从而使得预测的 Box 的位置和类别更接近于 Ground truth.

Query 可以视作代表不同 `Object` 的信息，而 Key 和 Value 可以视作代表图像的全局信息。

DETR 的 Decoder 主要有三个输入

1. Embedding Memory
1. Position Encoding
1. Object Queries

Object Queries 是一个可以学习的张量 $(100, b, 256)$, 数值类型是 `nn.Embedding`, Object Queries 矩阵内部通过学习建模了这 100 个物体之间的全局关系.

Object Queries 在 DETR 的具体实现中也充当位置编码的作用.

Where does the positional encoding of the first layer in Encoder come from

| Layer | Query | Key |
|:---:|:---:|:---:|
|1st multi-head self-attention | Positional Encoding | Positional Encoding |

Where does the positional encoding of each layer in Decoder come from

| Layer | Query | Key |
|:---:|:---:|:---:|
|1st multi-head self-attention | Object Queries | Object Queries |
|2nd multi-head self-attention | Object Queries | Positional Encoding |

Queries 可以视作代表不同 $Object$ 的信息, 而 Key 和 Value 可以视作代表图像的全局信息.

### 3.3 模型推理

<div>
  <img src="../assets/images/detr_outputs/detr_multi_attention.png" width="800"/>

  Fig. 3.4: Visualizing decoder attention for every predicted object (images from COCO val set). Predictions are made with DETR-DC5 model. Attention scores are coded with different colors for different objects. Decoder typically attends to object extremities, such as legs and heads. Best viewed in color.
</div>

<div>
  <img src="../assets/images/detr_outputs/detr_self_attention.png" width="800"/>

  Fig. 3.5: Encoder self-attention for a set of reference points. The encoder is able to separate individual instances. Predictions are made with baseline DETR model on a validation set image.

</div>

### 3.4 损失函数解读

\begin{aligned}
\hat{\sigma} &= \mathop{\arg\min}\limits_{\sigma(i)} \sum^N_i L_{\text{match}}\left(y_i, \hat{y}_{\sigma(i)}\right) \\
L_{\text{match}} &= - 1_\left\{{c_i \neq \emptyset}\right\} \hat{p}_{\sigma(i)}\left(c_i\right) + 1_\left\{{c_i \neq \emptyset}\right\} L_\text{box}\left(b_i, \hat{b}_{\sigma(i)}\right)
\end{aligned}

Decoder 的输出是维度为 $(b, 100, 256)$ 的张量。将其送入 2 个前馈网络 FFN 得到 classes $(b, 100, \text{num_classes} + 1)$ 和 Bounding Boxes $(b, 100, 4)$. 

目标检测任务就是输出无序集合预测问题 (Set prediction). 而无序集合如何与 GT Bounding Boxes 计算 loss?